In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB4 import *

learning_rate = 0.0001  
gamma = 1
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q3.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q3.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q3.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM8_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 27591, util : 0.941, makespan : 836
n_epi: 0, score : -5639.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 28761, util : 0.941, makespan : 839
n_episode: 1, score : -5899.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 27591, util : 0.941, makespan : 836
n_epi: 1, score : -5639.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 27965, util : 0.940, makespan : 831
n_episode: 2, score : -5836.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 27591, util : 0.941, makespan : 836
n_epi: 2, score : -5639.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 29349, util : 0.935, makespan : 855
n_episode: 3, score : -5919.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 27591, util

<ipython-input-1-b53ac6b66d60>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 27904, util : 0.941, makespan : 829
n_episode: 5, score : -5748.0, n_buffer : 2040, eps : 8.0%
--------------------------------------------------
flow time: 29273, util : 0.938, makespan : 825
n_epi: 5, score : -5586.0, n_buffer : 2040, eps : 0.0%
--------------------------------------------------
flow time: 30426, util : 0.925, makespan : 838
n_episode: 6, score : -5730.0, n_buffer : 2380, eps : 7.9%
--------------------------------------------------
flow time: 39207, util : 0.885, makespan : 685
n_epi: 6, score : -4811.0, n_buffer : 2380, eps : 0.0%
--------------------------------------------------
flow time: 39196, util : 0.883, makespan : 695
n_episode: 7, score : -5054.0, n_buffer : 2720, eps : 7.9%
--------------------------------------------------
flow time: 39310, util : 0.889, makespan : 688
n_epi: 7, score : -4749.0, n_buffer : 2720, eps : 0.0%
--------------------------------------------------
flow time: 38703, u

--------------------------------------------------
flow time: 42000, util : 0.875, makespan : 765
n_epi: 31, score : -5059.0, n_buffer : 10880, eps : 0.0%
--------------------------------------------------
flow time: 40658, util : 0.875, makespan : 782
n_episode: 32, score : -5484.0, n_buffer : 11220, eps : 7.7%
--------------------------------------------------
flow time: 43745, util : 0.881, makespan : 838
n_epi: 32, score : -5987.0, n_buffer : 11220, eps : 0.0%
--------------------------------------------------
flow time: 46585, util : 0.864, makespan : 848
n_episode: 33, score : -6266.0, n_buffer : 11560, eps : 7.7%
--------------------------------------------------
flow time: 46050, util : 0.866, makespan : 838
n_epi: 33, score : -5829.0, n_buffer : 11560, eps : 0.0%
--------------------------------------------------
flow time: 45261, util : 0.863, makespan : 866
n_episode: 34, score : -6280.0, n_buffer : 11900, eps : 7.7%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 34951, util : 0.876, makespan : 726
n_episode: 58, score : -5105.0, n_buffer : 20060, eps : 7.4%
--------------------------------------------------
flow time: 32621, util : 0.889, makespan : 699
n_epi: 58, score : -4921.0, n_buffer : 20060, eps : 0.0%
--------------------------------------------------
flow time: 33766, util : 0.884, makespan : 725
n_episode: 59, score : -4884.0, n_buffer : 20400, eps : 7.4%
--------------------------------------------------
flow time: 29223, util : 0.899, makespan : 717
n_epi: 59, score : -4901.0, n_buffer : 20400, eps : 0.0%
--------------------------------------------------
flow time: 31767, util : 0.884, makespan : 724
n_episode: 60, score : -5230.0, n_buffer : 20740, eps : 7.4%
--------------------------------------------------
flow time: 27271, util : 0.906, makespan : 712
n_epi: 60, score : -4852.0, n_buffer : 20740, eps : 0.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 37237, util : 0.892, makespan : 663
n_epi: 84, score : -4439.0, n_buffer : 28900, eps : 0.0%
--------------------------------------------------
flow time: 36720, util : 0.874, makespan : 685
n_episode: 85, score : -4701.0, n_buffer : 29240, eps : 7.2%
--------------------------------------------------
flow time: 33903, util : 0.895, makespan : 666
n_epi: 85, score : -4421.0, n_buffer : 29240, eps : 0.0%
--------------------------------------------------
flow time: 34769, util : 0.869, makespan : 701
n_episode: 86, score : -4956.0, n_buffer : 29580, eps : 7.1%
--------------------------------------------------
flow time: 37763, util : 0.890, makespan : 661
n_epi: 86, score : -4438.0, n_buffer : 29580, eps : 0.0%
--------------------------------------------------
flow time: 38101, util : 0.884, makespan : 676
n_episode: 87, score : -4748.0, n_buffer : 29920, eps : 7.1%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 29307, util : 0.906, makespan : 690
n_episode: 111, score : -4688.0, n_buffer : 38080, eps : 6.9%
--------------------------------------------------
flow time: 24441, util : 0.929, makespan : 686
n_epi: 111, score : -4305.0, n_buffer : 38080, eps : 0.0%
--------------------------------------------------
flow time: 25789, util : 0.925, makespan : 699
n_episode: 112, score : -4599.0, n_buffer : 38420, eps : 6.9%
--------------------------------------------------
flow time: 27095, util : 0.902, makespan : 680
n_epi: 112, score : -4501.0, n_buffer : 38420, eps : 0.0%
--------------------------------------------------
flow time: 29015, util : 0.889, makespan : 698
n_episode: 113, score : -4995.0, n_buffer : 38760, eps : 6.9%
--------------------------------------------------
flow time: 25631, util : 0.892, makespan : 736
n_epi: 113, score : -4956.0, n_buffer : 38760, eps : 0.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 27410, util : 0.902, makespan : 726
n_episode: 137, score : -4697.0, n_buffer : 46920, eps : 6.6%
--------------------------------------------------
flow time: 24325, util : 0.934, makespan : 703
n_epi: 137, score : -4194.0, n_buffer : 46920, eps : 0.0%
--------------------------------------------------
flow time: 25635, util : 0.914, makespan : 686
n_episode: 138, score : -4722.0, n_buffer : 47260, eps : 6.6%
--------------------------------------------------
flow time: 26150, util : 0.908, makespan : 667
n_epi: 138, score : -4287.0, n_buffer : 47260, eps : 0.0%
--------------------------------------------------
flow time: 27857, util : 0.899, makespan : 697
n_episode: 139, score : -4834.0, n_buffer : 47600, eps : 6.6%
--------------------------------------------------
flow time: 24895, util : 0.928, makespan : 666
n_epi: 139, score : -4168.0, n_buffer : 47600, eps : 0.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 34911, util : 0.872, makespan : 694
n_episode: 163, score : -4805.0, n_buffer : 55760, eps : 6.4%
--------------------------------------------------
flow time: 37632, util : 0.892, makespan : 653
n_epi: 163, score : -4522.0, n_buffer : 55760, eps : 0.0%
--------------------------------------------------
flow time: 37953, util : 0.884, makespan : 678
n_episode: 164, score : -4748.0, n_buffer : 56100, eps : 6.4%
--------------------------------------------------
flow time: 36281, util : 0.886, makespan : 663
n_epi: 164, score : -4474.0, n_buffer : 56100, eps : 0.0%
--------------------------------------------------
flow time: 36212, util : 0.892, makespan : 669
n_episode: 165, score : -4570.0, n_buffer : 56440, eps : 6.3%
--------------------------------------------------
flow time: 30137, util : 0.901, makespan : 678
n_epi: 165, score : -4413.0, n_buffer : 56440, eps : 0.0%
--------------------------------------------------
f

In [1]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB4 import *

learning_rate = 0.0001  
gamma = 1
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    params = torch.load("179q3.pt")
    q.load_state_dict(params)
    q.eval()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(179, 1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q3.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q3.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q3.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM8_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 24011, util : 0.916, makespan : 691
n_episode: 179, score : -4546.0, n_buffer : 340, eps : 6.2%
--------------------------------------------------
flow time: 23972, util : 0.928, makespan : 687
n_epi: 179, score : -4293.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 26759, util : 0.907, makespan : 735
n_episode: 180, score : -4995.0, n_buffer : 680, eps : 6.2%
--------------------------------------------------
flow time: 23972, util : 0.928, makespan : 687
n_epi: 180, score : -4293.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 24743, util : 0.921, makespan : 730
n_episode: 181, score : -4618.0, n_buffer : 1020, eps : 6.2%
--------------------------------------------------
flow time: 23972, util : 0.928, makespan : 687
n_epi: 181, score : -4293.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 

<ipython-input-1-4c095b892395>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 25109, util : 0.919, makespan : 724
n_episode: 184, score : -4737.0, n_buffer : 2040, eps : 6.2%
--------------------------------------------------
flow time: 26493, util : 0.921, makespan : 763
n_epi: 184, score : -5190.0, n_buffer : 2040, eps : 0.0%
--------------------------------------------------
flow time: 27740, util : 0.912, makespan : 783
n_episode: 185, score : -5312.0, n_buffer : 2380, eps : 6.2%
--------------------------------------------------
flow time: 25765, util : 0.913, makespan : 740
n_epi: 185, score : -4620.0, n_buffer : 2380, eps : 0.0%
--------------------------------------------------
flow time: 27061, util : 0.892, makespan : 736
n_episode: 186, score : -5002.0, n_buffer : 2720, eps : 6.1%
--------------------------------------------------
flow time: 26879, util : 0.903, makespan : 736
n_epi: 186, score : -4848.0, n_buffer : 2720, eps : 0.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 24437, util : 0.930, makespan : 699
n_epi: 210, score : -4500.0, n_buffer : 10880, eps : 0.0%
--------------------------------------------------
flow time: 25172, util : 0.911, makespan : 732
n_episode: 211, score : -5143.0, n_buffer : 11220, eps : 5.9%
--------------------------------------------------
flow time: 33073, util : 0.879, makespan : 685
n_epi: 211, score : -4553.0, n_buffer : 11220, eps : 0.0%
--------------------------------------------------
flow time: 32622, util : 0.879, makespan : 697
n_episode: 212, score : -4784.0, n_buffer : 11560, eps : 5.9%
--------------------------------------------------
flow time: 23864, util : 0.927, makespan : 725
n_epi: 212, score : -4621.0, n_buffer : 11560, eps : 0.0%
--------------------------------------------------
flow time: 25123, util : 0.908, makespan : 719
n_episode: 213, score : -4935.0, n_buffer : 11900, eps : 5.9%
--------------------------------------------------
f

--------------------------------------------------
flow time: 23826, util : 0.930, makespan : 709
n_epi: 236, score : -4631.0, n_buffer : 19720, eps : 0.0%
--------------------------------------------------
flow time: 24272, util : 0.922, makespan : 724
n_episode: 237, score : -4884.0, n_buffer : 20060, eps : 5.6%
--------------------------------------------------
flow time: 24392, util : 0.927, makespan : 711
n_epi: 237, score : -4704.0, n_buffer : 20060, eps : 0.0%
--------------------------------------------------
flow time: 24854, util : 0.918, makespan : 752
n_episode: 238, score : -4825.0, n_buffer : 20400, eps : 5.6%
--------------------------------------------------
flow time: 24567, util : 0.927, makespan : 728
n_epi: 238, score : -4565.0, n_buffer : 20400, eps : 0.0%
--------------------------------------------------
flow time: 24550, util : 0.928, makespan : 714
n_episode: 239, score : -4518.0, n_buffer : 20740, eps : 5.6%
--------------------------------------------------
f

...

--------------------------------------------------
flow time: 25007, util : 0.948, makespan : 797
n_epi: 886, score : -5635.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25942, util : 0.947, makespan : 825
n_episode: 887, score : -5893.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23392, util : 0.929, makespan : 723
n_epi: 887, score : -4732.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23392, util : 0.929, makespan : 723
n_episode: 888, score : -4732.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23257, util : 0.934, makespan : 747
n_epi: 888, score : -4542.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23149, util : 0.929, makespan : 713
n_episode: 889, score : -4688.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 25723, util : 0.926, makespan : 737
n_epi: 912, score : -4830.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 26187, util : 0.928, makespan : 762
n_episode: 913, score : -4486.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23661, util : 0.939, makespan : 735
n_epi: 913, score : -4791.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24001, util : 0.941, makespan : 732
n_episode: 914, score : -4572.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 914, score : -4686.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23837, util : 0.941, makespan : 721
n_episode: 915, score : -4488.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 24213, util : 0.929, makespan : 742
n_epi: 938, score : -4584.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24459, util : 0.919, makespan : 740
n_episode: 939, score : -4760.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23889, util : 0.931, makespan : 718
n_epi: 939, score : -4579.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23756, util : 0.939, makespan : 722
n_episode: 940, score : -4334.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23727, util : 0.937, makespan : 724
n_epi: 940, score : -4332.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24625, util : 0.925, makespan : 754
n_episode: 941, score : -4800.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 23969, util : 0.910, makespan : 724
n_epi: 964, score : -4542.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23911, util : 0.917, makespan : 740
n_episode: 965, score : -5072.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23624, util : 0.934, makespan : 692
n_epi: 965, score : -4809.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23238, util : 0.925, makespan : 716
n_episode: 966, score : -4641.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24153, util : 0.917, makespan : 749
n_epi: 966, score : -4703.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24439, util : 0.918, makespan : 743
n_episode: 967, score : -4641.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 25667, util : 0.927, makespan : 742
n_epi: 990, score : -4822.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25997, util : 0.920, makespan : 705
n_episode: 991, score : -4545.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24153, util : 0.917, makespan : 749
n_epi: 991, score : -4703.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24052, util : 0.916, makespan : 693
n_episode: 992, score : -4466.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23585, util : 0.934, makespan : 711
n_epi: 992, score : -4636.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24151, util : 0.911, makespan : 717
n_episode: 993, score : -4748.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------